# 🧪 Laboratório 1 — Publicação de Post em Blog com **Multiagentes** (CrewAI)

**Objetivo do Lab:** aprender a **modelar papéis** (agentes), **definir tarefas** com entradas/saídas claras e **orquestrar** um *mini pipeline* multiagente para produzir um post de blog, do planejamento à edição final.

> Ritmo: **conceito → exemplo → código guiado → execução → variações**


## 🔑 Como usar um modelo LLM (Groq) — chave e configuração rápida

Este lab usa o ecossistema **OpenAI-compatible**. Você pode usar **Groq** como provedor de LLM.

1. Crie sua conta e gere uma chave em: https://console.groq.com  
2. No Colab, defina **variáveis de ambiente** para o provedor compatível:
   - `OPENAI_API_KEY`: sua chave da Groq
   - `OPENAI_API_BASE`: endpoint da Groq → `https://api.groq.com/openai/v1`

> Com isso, os agentes/tarefas do CrewAI já usarão a Groq automaticamente.


## 🔧 Passo 0 — Instalar dependências
Execute a célula abaixo. Se o Colab pedir para reiniciar, aceite e rode novamente esta célula.


In [ ]:
!pip -q install crewai crewai-tools langchain python-dotenv


## 🔐 Passo 1 — Configurar chaves no ambiente (Groq ou outro provedor compatível)

> Se preferir, você pode **colar as chaves diretamente** abaixo (apenas para este notebook).


In [3]:
import os

# ===> DESCOMENTE e COLE suas chaves para usar a Groq (OpenAI-compatible) <===
# os.environ["OPENAI_API_KEY"] = "SUA_CHAVE_DA_GROQ_AQUI"
# os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"

print("OPENAI_API_KEY set?", "OPENAI_API_KEY" in os.environ)
print("OPENAI_API_BASE set?", "OPENAI_API_BASE" in os.environ)


OPENAI_API_KEY set? True
OPENAI_API_BASE set? False


## 🧠 Passo 2 — Conceitos essenciais (rapidinho)

- **Agent** (*Agente*): papel com **objetivo** e **backstory**.
- **Task** (*Tarefa*): instrução **específica** com **expected_output**.
- **Crew**: orquestra agentes + tarefas.
- **Inputs**: variáveis **injetadas** (ex.: `{topic}`).


In [4]:
from crewai import Agent, Task, Crew, Process
print("CrewAI importado com sucesso.")


CrewAI importado com sucesso.


## ✍️ Passo 4 — Defina o **tema** do post (input do pipeline)


In [5]:
topic = "Inteligência Artificial aplicada ao aprendizado de música"
topic


'Inteligência Artificial aplicada ao aprendizado de música'

## 🧩 Passo 5 — Crie os **Agentes** (papéis) com objetivos e backstory


In [6]:
planner = Agent(
    role="Planejador de Conteúdo",
    goal=f"Planejar conteúdo envolvente e factualmente preciso sobre {topic}",
    backstory=(
        "Você está trabalhando no planejamento de um artigo de blog "
        f"sobre o tema: {topic}. "
        "Você coleta informações que ajudam o público a aprender algo "
        "e tomar decisões informadas. Seu trabalho é a base para que o "
        "Redator de Conteúdo escreva um artigo sobre esse tema."
    ),
    allow_delegation=False,
    verbose=True
)



In [7]:
writer = Agent(
    role="Redator de Conteúdo",
    goal=f"Escrever um artigo opinativo perspicaz e factualmente preciso sobre o tema: {topic}",
    backstory=(
        "Você está trabalhando na redação de um novo artigo opinativo "
        f"sobre o tema: {topic}. "
        "Você baseia sua escrita no trabalho do Planejador de Conteúdo, "
        "que fornece um esboço e contexto relevante sobre o tema. "
        "Você segue os principais objetivos e a direção do esboço, "
        "conforme fornecido pelo Planejador de Conteúdo. "
        "Você também fornece percepções objetivas e imparciais e as fundamenta "
        "com informações fornecidas pelo Planejador de Conteúdo. "
        "Você reconhece no seu artigo opinativo quando suas declarações são "
        "opiniões em oposição a declarações objetivas."
    ),
    allow_delegation=False,
    verbose=True
)

In [8]:
editor = Agent(
    role="Editor",
    goal="Editar um post de blog para alinhar com o estilo de escrita da organização.",
    backstory=(
        "Você é um editor que recebe um post de blog do Redator de Conteúdo. "
        "Seu objetivo é revisar o post de blog para garantir que ele siga as "
        "melhores práticas jornalísticas, ofereça pontos de vista equilibrados "
        "ao apresentar opiniões ou afirmações, e também evite tópicos ou opiniões "
        "altamente controversos sempre que possível."
    ),
    allow_delegation=False,
    verbose=True
)

## 🗂️ Passo 6 — Crie as **Tarefas** (descrições + expected_output)


In [9]:
plan = Task(
    description=(
        "1. Priorizar as tendências mais recentes, principais atores e notícias relevantes sobre o tema.\n"
        "2. Identificar o público-alvo, considerando seus interesses e pontos de dor.\n"
        "3. Desenvolver um esboço de conteúdo detalhado incluindo uma introdução, pontos principais e uma chamada para ação.\n"
        "4. Incluir palavras-chave de SEO e dados ou fontes relevantes."
    ),
    expected_output=(
        "Um documento de plano de conteúdo abrangente com esboço, análise do público, "
        "palavras-chave de SEO e recursos."
    ),
    agent=planner,
)

In [10]:
write = Task(
    description=(
        "1. Usar o plano de conteúdo para criar um post de blog envolvente sobre o tema.\n"
        "2. Incorporar palavras-chave de SEO de forma natural.\n"
        "3. As seções/subtítulos devem estar nomeados adequadamente de forma atraente.\n"
        "4. Garantir que o post esteja estruturado com uma introdução envolvente, corpo com boas ideias e uma conclusão resumida.\n"
        "5. Revisar para corrigir erros gramaticais e alinhar com a voz da marca."
    ),
    expected_output=(
        "Um post de blog bem escrito em formato markdown, pronto para publicação, "
        "cada seção deve conter 2 ou 3 parágrafos."
    ),
    agent=writer,
)

In [11]:

edit = Task(
    description=(
        "Revisar o post de blog fornecido para corrigir erros gramaticais, "
        "alinhar com a voz da marca, traduzir para o português brasileiro "
        "e garantir que a formatação markdown esteja correta e pronta para publicação."
    ),
    expected_output=(
        "Um post de blog bem escrito em formato markdown, pronto para publicação, "
        "cada seção deve conter 2 ou 3 parágrafos, com a formatação markdown correta."
    ),
    agent=editor
)

## 🧪 Passo 7 — Monte o **Crew** e execute


In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=False  # coloque True para ver logs detalhados
)

result = crew.kickoff(inputs={"topic": topic})


## 👀 Passo 8 — Visualize o resultado em Markdown


In [ ]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

## 🔁 Variações sugeridas
- Mude `topic` para um assunto da sua área (marketing, manufatura, saúde).  
- Peça ao **Editor** para sugerir **título SEO** e **meta descrição** adicionais.  
- Adicione uma nova tarefa de **Fact-Checking** com uma ferramenta de busca (ex.: `SerperDevTool`).
